In [142]:
#imports
import math
import numpy as np
import random
import itertools
from random import choices
import cmath
BOLD = '\033[1m'
RESET = '\033[0m'


In [143]:
#constants
GAIN_MATRIX = (
    (3.95370716e-13, 1.40831353e-12, 1.30345221e-13, 1.58616128e-12),
    (1.10454943e-11, 1.08797904e-13, 1.70626457e-11, 1.87910752e-13),
    (5.35349571e-12, 1.75915771e-12, 1.10933550e-11, 2.15002508e-12),
    (9.57092193e-12, 1.14512900e-11, 7.62246884e-13, 1.58625393e-11)
)

SUM_GAIN = (
    2.63652827e-11, 1.47275591e-11, 2.90485928e-11, 1.97866364e-11
)

D=10**6  #bandwidth
N=-174 + 10 * math.log10(D) #noise in dbm

In [144]:
#generating population in dbm
def generate_random_solutions(powers, no_of_users, num_random_solutions):
    return np.random.choice(powers, size=(num_random_solutions, no_of_users))


In [145]:
#usage
powers = [0,2,4]
no_of_users = 4
num_random_solutions = 8
print(f"{BOLD}POWERS IN DBM{RESET}")
pop_size_dbm = generate_random_solutions(powers, no_of_users, num_random_solutions)
print(pop_size_dbm)

POWERS IN DBM
[[2 0 0 0]
 [0 0 2 0]
 [4 4 2 4]
 [2 4 0 0]
 [4 0 0 2]
 [0 0 2 4]
 [4 0 4 2]
 [4 4 4 4]]


In [146]:
# converting population to linear watt
def calculate_linear_trans_power(pop_size_dbm):
    result=(pow(10, -3) * np.power(10.0, np.array(pop_size_dbm, dtype="float") / 10))
    return result


In [147]:
#usage
print(f"{BOLD}POWERS IN LINEAR{RESET}")
linear_pop=calculate_linear_trans_power(pop_size_dbm)
print(linear_pop)

POWERS IN LINEAR
[[0.00158489 0.001      0.001      0.001     ]
 [0.001      0.001      0.00158489 0.001     ]
 [0.00251189 0.00251189 0.00158489 0.00251189]
 [0.00158489 0.00251189 0.001      0.001     ]
 [0.00251189 0.001      0.001      0.00158489]
 [0.001      0.001      0.00158489 0.00251189]
 [0.00251189 0.001      0.00251189 0.00158489]
 [0.00251189 0.00251189 0.00251189 0.00251189]]


In [148]:
def sum_linear_powers(array):
    return np.sum(array, axis=1)

In [149]:
# usage
print(f"{BOLD}SUM OF LINEAR POWERS BY ROW{RESET}")
row_sums = sum_linear_powers(linear_pop)
print(*row_sums, sep='\n')

SUM OF LINEAR POWERS BY ROW
0.0045848931924611135
0.0045848931924611135
0.009120552486989855
0.006096779623970694
0.006096779623970694
0.006096779623970694
0.007608666055480274
0.010047545726038321


In [150]:
#usage for converting the noise to linear
linear_noise = calculate_linear_trans_power([N])
print("Linear noise:", linear_noise)

Linear noise: [3.98107171e-15]


In [151]:
def calculate_formula_and_sums(linear_pop, SUM_GAIN, linear_noise):
    a = np.array(linear_pop)
    b = np.array(SUM_GAIN)
    bit_rate_matrix = (D * np.log2(1 + ((a * b) / linear_noise)))
    return bit_rate_matrix

In [152]:
#Usage for calculating the bitrates through big formula
print(f"{BOLD}BITRATES(formula){RESET}")
bitrate=calculate_formula_and_sums(linear_pop, SUM_GAIN, linear_noise)
print(bitrate)

BITRATES(formula)
[[3523086.17671602 2232475.21650449 3052533.55366803 2577774.03041514]
 [2930294.44975843 2232475.21650449 3651275.96911754 2577774.03041514]
 [4140399.83799027 3363516.1582802  3651275.96911754 3753232.64345308]
 [3523086.17671602 3363516.1582802  3052533.55366803 2577774.03041514]
 [4140399.83799027 2232475.21650449 3052533.55366803 3150104.99872635]
 [2930294.44975843 2232475.21650449 3651275.96911754 3753232.64345308]
 [4140399.83799027 2232475.21650449 4272652.02625749 3150104.99872635]
 [4140399.83799027 3363516.1582802  4272652.02625749 3753232.64345308]]


In [153]:
def row_wise_sums(matrix):
    return [sum(row) for row in matrix]

In [154]:
# USAGE
print(f"{BOLD}SUM OF BITRATES{RESET}")
row_sum_bitrate = row_wise_sums(bitrate)
print(*row_sum_bitrate, sep='\n')


SUM OF BITRATES
11385868.97730368
11391819.665795594
14908424.608841093
12516909.91907939
12575513.606889138
12567278.278833544
13795632.079478601
15529800.665981045


In [155]:
# for my understanding

# Printing row_sums and row_sum_bitrate side by side
print(f"{BOLD}ROW SUMS (Linear)    vs     ROW SUM BITRATES{RESET}")
for i in range(len(row_sums)):
    print(f"{row_sums[i]:<15}          {row_sum_bitrate[i]:<15}")


ROW SUMS (Linear)    vs     ROW SUM BITRATES
0.0045848931924611135          11385868.97730368
0.0045848931924611135          11391819.665795594
0.009120552486989855          14908424.608841093
0.006096779623970694          12516909.91907939
0.006096779623970694          12575513.606889138
0.006096779623970694          12567278.278833544
0.007608666055480274          13795632.079478601
0.010047545726038321          15529800.665981045


In [156]:
def sort_row_sums(array):
    return np.sort(array)

In [157]:
print(f"{BOLD}SORTED ROW SUMS IN INCREASING ORDER OF LINEAR POWER{RESET}")
sorted_array_linear_power = sort_row_sums(row_sums)
print(*sorted_array_linear_power, sep='\n')  # Output: [1 3 5 7 9]

SORTED ROW SUMS IN INCREASING ORDER OF LINEAR POWER
0.0045848931924611135
0.0045848931924611135
0.006096779623970694
0.006096779623970694
0.006096779623970694
0.007608666055480274
0.009120552486989855
0.010047545726038321


In [158]:
print(f"{BOLD}SORTED ROW SUMS IN INCREASING ORDER OF BITRATES{RESET}")
sorted_array_bitrates = sort_row_sums(row_sum_bitrate)
print(*sorted_array_bitrates, sep='\n')  # Output: [1 3 5 7 9]

SORTED ROW SUMS IN INCREASING ORDER OF BITRATES
11385868.97730368
11391819.665795594
12516909.91907939
12567278.278833544
12575513.606889138
13795632.079478601
14908424.608841093
15529800.665981045


In [167]:
# for my understanding

# Printing row_sums and row_sum_bitrate side by side
print(f"{BOLD}Increasing order ROW SUMS Linear    vs        Increasing order ROW SUM BITRATES{RESET}")
for i in range(len(row_sums)):
    print(f"{sorted_array_linear_power[i]:<15}           {sorted_array_bitrates[i]:<15}")

Increasing order ROW SUMS Linear    vs        Increasing order ROW SUM BITRATES
0.0045848931924611135           11385868.97730368
0.0045848931924611135           11391819.665795594
0.006096779623970694           12516909.91907939
0.006096779623970694           12567278.278833544
0.006096779623970694           12575513.606889138
0.007608666055480274           13795632.079478601
0.009120552486989855           14908424.608841093
0.010047545726038321           15529800.665981045


In [168]:
def select_first_half(array):
    midpoint = len(array) // 2
    return array[:midpoint]


In [170]:
first_half_linear_power_increasing = select_first_half(sorted_array_linear_power)
print(f"{BOLD}TOP HALF OF SORTED ROW SUMS OF LINEAR POWER{RESET}")
print(*first_half_linear_power_increasing, sep='\n') 
 

TOP HALF OF SORTED ROW SUMS OF LINEAR POWER
0.0045848931924611135
0.0045848931924611135
0.006096779623970694
0.006096779623970694


In [172]:
first_half_bitrates_increasing = select_first_half(sorted_array_bitrates)
print(f"{BOLD}TOP HALF OF SORTED ROW SUMS OF BITRATES{RESET}")
print(*first_half_bitrates_increasing, sep='\n') 

TOP HALF OF SORTED ROW SUMS OF BITRATES
11385868.97730368
11391819.665795594
12516909.91907939
12567278.278833544


In [ ]:
def get_initial_population(bitrate, pop_size_dbm, sorted_row_sums):
    half_sorted_row_sums = select_half_of_sorted_sums(sorted_row_sums)

    initial_population = []
    for sorted_sum in half_sorted_row_sums:
        index = row_wise_sums(bitrate).index(sorted_sum)
        initial_population.append(pop_size_dbm[index])

    return initial_population



In [ ]:
# Call the function and get the initial population
initial_population = get_initial_population(bitrate, pop_size_dbm, sorted_row_sums)

# Print the initial population
print("Initial population:")
for solution in initial_population:
    print(solution)

In [ ]:
def single_point_crossover(parent1, parent2):
    # Perform single-point crossover
    crossover_point = random.randint(1, len(parent1) - 1)
    child1 = np.hstack((parent1[:crossover_point], parent2[crossover_point:]))
    child2 = np.hstack((parent2[:crossover_point], parent1[crossover_point:]))
    
    return child1, child2



In [ ]:
def perform_single_point_crossover(initial_population):
    offspring = []
    for i in range(0, len(initial_population), 2):
        parent1 = initial_population[i]
        parent2 = initial_population[i + 1]

        child1, child2 = single_point_crossover(parent1, parent2)

        offspring.append(child1)
        offspring.append(child2)

    return offspring

In [ ]:
# usage
# Call the perform_single_point_crossover function and print the offspring
offspring = perform_single_point_crossover(initial_population)

print("Offspring after single-point crossover:")
for child in offspring:
    print(child)


In [ ]:
def mutate_one_gene_per_individual(population, powers):
    mutated_population = np.copy(population)

    for i, individual in enumerate(mutated_population):
        # Choose a random gene within the individual
        gene_index = random.randint(0, len(individual) - 1)

        # Store the original power level of the chosen gene
        original_power = individual[gene_index]

        # Mutate the chosen gene with a random power level from the powers list
        individual[gene_index] = random.choice(powers)

        # Print the mutation information
#         print(f"Offspring {i + 1}: Gene {gene_index + 1} was mutated from {original_power} to {individual[gene_index]}")

    return mutated_population


In [ ]:
# usage
mutated_offspring = mutate_one_gene_per_individual(offspring, powers)

print("\nMutated offspring:")
for child in mutated_offspring:
    print(child)
    
print(type(mutated_offspring))


In [ ]:
def print_stacked_arrays(array_list1, array_list2):
    stacked_array = np.vstack((array_list1, array_list2))
    for row in stacked_array:
        print(row)


In [ ]:
#usage
print_stacked_arrays(initial_population, mutated_offspring)